## Main

**Ptf :** 

- **20% projet**  
- **15% obligataire**   
- **20% fonds à impact / mirova**  
- **5% ETF**  
- **5% crypto verte/entreprise qui investit dans manière verte de gérer gestion de données**  
- **15% oblig corpo**  
- **10% actions** 

**Code :** 

- **Recherche des données**  (data_collector.py)
- **Analyse des données** (data_analysis.py)
- **Construction du portefeuille** (strategy.py)  
- **Reporting** 
- **Suivi du ptf=\> mécanisme de sortie en cas de faillite \=\> stress tests**

In [5]:
from fredapi import Fred
import importlib
import data_collector
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import timedelta  
import importlib
import matplotlib.pyplot as plt
from tqdm import tqdm

importlib.reload(data_collector)

<module 'data_collector' from '/Users/cosima/Desktop/Finance durable/projet fin durable/data_collector.py'>

# 1. Récupération des données avec data_collector
## module de collecte et de traitement des données

In [145]:
class YahooFinanceDataCollector:
    """Classe pour récupérer et traiter les données financières de Yahoo Finance."""

    def __init__(self, tickers, start_date, end_date, output_dir="financial_data"):
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)

    def download_data(self):
        """Télécharge les données depuis Yahoo Finance."""
        data = yf.download(self.tickers, start=self.start_date, end=self.end_date)
        return data['Adj Close'] if 'Adj Close' in data.columns else data['Close']

    def handle_missing_values(self, df):
        """Remplit les valeurs manquantes en utilisant forward-fill uniquement."""
        return df.ffill().bfill()

    def remove_outliers(self, df):
        """Supprime les valeurs aberrantes avec le Z-score."""
        z_scores = np.abs(stats.zscore(df, nan_policy='omit'))
        df[z_scores > 3] = np.nan
        return self.handle_missing_values(df)

    def save_to_csv(self, df, filename="data_actifs.csv"):
        """Sauvegarde les données au format CSV."""
        filepath = os.path.join(self.output_dir, filename)
        df.to_csv(filepath)
        print(f"Données sauvegardées dans {filepath}")

    def run(self):
        """Exécute toutes les étapes du pipeline."""
        print("Téléchargement des données Yahoo Finance...")
        data = self.download_data()

        print("Traitement des valeurs manquantes...")
        data = self.handle_missing_values(data)

        print("Suppression des valeurs aberrantes...")
        data = self.remove_outliers(data)

        self.save_to_csv(data)

        return data

In [154]:
# Définition des tickers financiers à chercher

tickers = ['CYBO.L','7951.T','SW.PA','CAP.PA','EL.PA', '0P0000G6X1.F', 'XDG6.L', 'CYBO.SW', '0P0001IVQQ.F',
           '0P0001HZQR.F','0P00016ZNX.F','NRJ.PA','CYBO.SW','XCH-EUR','EQIX','VIE.PA','CAP.PA','IBE.MC','OVH.PA',
           'SAP','ENEL.MI','VOLV-B.ST','0P00018AOI.F','0P000155QH.F','0P0001NHSI.F','0P000100RL.F','0P00000DLW.F',
           'ECN.PA','EMEC.DE','XZRE.L','0P0000G6X1.F','0P0000KU3M.F','SIGN.SW','TOM.OL','0P000111I9.F', '2353.TW']
# Période de téléchargement
start_date = '2000-01-01'
end_date = '2024-12-31'

#Importations des objets de collecte de données de data_collector
#Création de deux dataframes df_yahoo et df_fred
#from data_collector import YahooFinanceDataCollector, FredDataCollector
# Collecte des données financières depuis Yahoo Finance
yahoo_collector = YahooFinanceDataCollector(tickers, start_date, end_date)
df_yahoo = yahoo_collector.run()

[***                    6%                       ]  2 of 33 completed

Téléchargement des données Yahoo Finance...


[*********************100%***********************]  33 of 33 completed


Traitement des valeurs manquantes...
Suppression des valeurs aberrantes...
Données sauvegardées dans financial_data/data_actifs.csv


In [149]:
df_yahoo

Ticker,0P00000DLW.F,0P0000G6X1.F,0P000100RL.F,0P000155QH.F,0P00016ZNX.F,0P00018AOI.F,0P0001HZQR.F,0P0001IVQQ.F,0P0001NHSI.F,7951.T,...,IBE.MC,NRJ.PA,OVH.PA,SAP,SW.PA,VIE.PA,VOLV-B.ST,XCH-EUR,XDG6.L,XZRE.L
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,235.926620,1403.729980,116.260002,128.160004,102.620003,10188.299805,100.000000,100.000000,89.410004,185.509491,...,1.100288,39.958252,19.704,36.803883,11.920436,12.968624,16.815487,720.202454,33.822800,29.7862
2000-01-04,235.926620,1403.729980,116.260002,128.160004,102.620003,10188.299805,100.000000,100.000000,89.410004,185.509491,...,1.083070,39.958252,19.704,33.531437,11.210885,12.968624,16.428923,720.202454,33.822800,29.7862
2000-01-05,235.926620,1403.729980,116.260002,128.160004,102.620003,10188.299805,100.000000,100.000000,89.410004,187.208832,...,1.081430,39.958252,19.704,32.321068,11.090264,12.968624,16.428923,720.202454,33.822800,29.7862
2000-01-06,235.926620,1403.729980,116.260002,128.160004,102.620003,10188.299805,100.000000,100.000000,89.410004,187.208832,...,1.081430,39.958252,19.704,32.186588,11.487612,12.968624,16.428923,720.202454,33.822800,29.7862
2000-01-07,235.926620,1403.729980,116.260002,128.160004,102.620003,10188.299805,100.000000,100.000000,89.410004,192.023605,...,1.090449,39.958252,19.704,43.034988,11.317320,12.968624,16.506233,720.202454,33.822800,29.7862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-26,253.375412,1807.680054,146.119995,150.570007,94.400002,10656.830078,139.460007,113.419998,104.639999,1100.232788,...,12.919088,26.302000,8.545,213.919998,78.400002,26.799999,256.631012,21.805870,36.502499,29.7875
2024-12-27,255.978104,1817.069946,146.119995,152.279999,94.489998,10656.830078,140.639999,113.849998,104.639999,1119.031738,...,12.943667,26.011999,8.665,213.919998,78.849998,26.940001,258.751923,20.464497,36.715000,29.7875
2024-12-28,255.978104,1817.069946,146.119995,152.279999,94.489998,10656.830078,140.639999,113.849998,104.639999,1119.031738,...,12.943667,26.011999,8.665,213.919998,78.849998,26.940001,258.751923,20.222158,36.715000,29.7875


# Entreprises selon critères ESG
### selection actions et obligations corpo

In [120]:
import pandas as pd

# Données structurées : entreprises et les indices/labels auxquels elles appartiennent
MSCI_World_Low_Carbon_SRI = ["MSFT", "NVDA", "GOOGL", "TSLA", "LLY", "V", "MA", "HD", "JNJ"]
MSCI_Europe_Low_Carbon_SRI = ["SAP", "NVO", "NVS", "HSBC", "SU.PA", "AI.PA", "RELX", "RMS.PA", "OR.PA", "ZURVY"]
MSCI_World_ESG_Leaders = ["SAMPO.HE", "WPM", "EA", "ZURN.SW", "HIG", "CBOE", "CAH", "BN.PA", "LNG", "SREN.SW"]
FTSE4Good = ["AAPL", "MSFT", "NVDA", "GOOGL", "AVGO", "GOOG", "LLY", "V", "MA", "UNH"]
SP_Clean_Energy = ["IBE", "SSE", "VWS", "FSLR", "ENPH", "600900.SS", "EDP", "NXT", "EQTL3.SA", "ORSTED"]
SP_500_ESG_Leaders = ["AAPL", "NVDA", "MSFT", "TSLA", "GOOGL", "V", "XOM", "UNH", "MA"]
SP_SmallCap_600 = ["COOP", "EAT", "JXN", " MMSI", "AWI", "LNC", "SPXC", "ACIW", "JBT", "BMI"]
SP_MidCap_400 = ["WSO", "EME", "FNF", "EQH", "SFM", "CSL", "USFD", "BJ", "BURL", "RPM"]
SP_EUROUSA_50 = ["SAN", "BNP", "BBVA", "ABBV", "ENGI", "DB1", "ISP", "KBC", "ALV", "DTE"]
SP_500_Equal_Weight_ESG = ["LUV", "ULTA", "NEM", "T", "INTC", "FIS", "COR", "MOS", "DAY", "UNH"]
SP_Water_Decrement = ["AWK", "XYL", "SVT", "VLTO", "UU.L", "SBSP3.SA", "WTRG", "VIE.PA", "GEBN.SW", "ECL"]
SP_Water_Index = ["AWK", "XYL", "SVT", "VLTO", "UUGRY", "SBS", "WTRG", "VEOEY", "GBERF", "ECL"]
CSR_Hub_Better_World_Companies_AND_100_Low_Carbon_Pioneers = [
    "MMM", "ABB", "AA", "AABA", "AEP", "BAYN.DE", "BTI", "DELL", "CTVA", "EQNR", 
    "HMC", "HSBC", "ITRI", "MKS.L", "PEP", "SIEGY", "GS", "UL", "WMT"
]
Sustainalytics_top_ESG= ["ACKB.BR", "ADEN.SW", "ALX.AX", "AVT", "BGN.MI", "COV.PA", "7912.T", "DXS.AX", 
                                  "HAS", "COL.MC", "LEG.DE", "PSN.L", "PGEO.JK", "RAND.AS", "RDF.JO", "RS1.L", 
                                  "SGRO.L", "TEG.DE", "THULE.ST", "TPZ.TO", "TCL.AX", "TFPM", "BBOX.L", "URW.AS", 
                                  "VAKRANGEE.NS", "VCX.AX", "VNA.DE", "WPM"]

# Liste des catégories et leurs entreprises
categories_dict = {
    "MSCI World Low Carbon SRI": MSCI_World_Low_Carbon_SRI,
    "MSCI Europe Low Carbon SRI": MSCI_Europe_Low_Carbon_SRI,
    "MSCI World ESG Leaders": MSCI_World_ESG_Leaders,
    "FTSE4Good": FTSE4Good,
    "S&P Clean Energy": SP_Clean_Energy,
    "S&P 500 ESG Leaders": SP_500_ESG_Leaders,
    "S&P SmallCap 600": SP_SmallCap_600,
    "S&P MidCap 400": SP_MidCap_400,
    "S&P EUROUSA 50": SP_EUROUSA_50,
    "S&P 500 Equal Weight ESG+": SP_500_Equal_Weight_ESG,
    "S&P Water Decrement": SP_Water_Decrement,
    "S&P Water Index": SP_Water_Index,
    "CSR Hub Better World Companies AND 100 Low Carbon Pioneers": CSR_Hub_Better_World_Companies_AND_100_Low_Carbon_Pioneers,
    "Sustainalytics top ESG": Sustainalytics_top_ESG
}

# Liste des entreprises uniques
noms = list(set(
    MSCI_World_Low_Carbon_SRI + MSCI_Europe_Low_Carbon_SRI + MSCI_World_ESG_Leaders + 
    FTSE4Good + SP_Clean_Energy + SP_500_ESG_Leaders + SP_SmallCap_600 + 
    SP_MidCap_400 + SP_EUROUSA_50 + SP_500_Equal_Weight_ESG + SP_Water_Decrement + 
    SP_Water_Index + CSR_Hub_Better_World_Companies_AND_100_Low_Carbon_Pioneers
    + Sustainalytics_top_ESG
))

# Initialisation du DataFrame avec les entreprises
data = {"Entreprise": noms}

# Remplir les colonnes avec des croix pour les entreprises qui appartiennent à chaque catégorie
for category, companies in categories_dict.items():
    data[category] = ["✓" if ent in companies else "" for ent in noms]

# Créer le DataFrame final
df = pd.DataFrame(data)
# Save updated CSV
updated_file_path = "ESG_Indices_and_Labels_Ticker.csv"
df.to_csv(updated_file_path, index=False)


In [121]:
df

,Entreprise,MSCI World Low Carbon SRI,MSCI Europe Low Carbon SRI,MSCI World ESG Leaders,FTSE4Good,S&P Clean Energy,S&P 500 ESG Leaders,S&P SmallCap 600,S&P MidCap 400,S&P EUROUSA 50,S&P 500 Equal Weight ESG+,S&P Water Decrement,S&P Water Index,CSR Hub Better World Companies AND 100 Low Carbon Pioneers,Sustainalytics top ESG
0,JXN,,,,,,,✓,,,,,,,
1,IBE,,,,,✓,,,,,,,,,
2,WTRG,,,,,,,,,,,✓,✓,,
3,MSFT,✓,,,✓,,✓,,,,,,,,
4,ITRI,,,,,,,,,,,,,✓,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,ZURN.SW,,,✓,,,,,,,,,,,
138,COR,,,,,,,,,,✓,,,,
139,VWS,,,,,✓,,,,,,,,,
140,GOOG,,,,✓,,,,,,,,,,


In [129]:
def entreprises_multiples_categories(df, seuil=2):
    """
    Renvoie les entreprises qui apparaissent dans au moins 'seuil' catégories.
    
    Args:
        df (pd.DataFrame): Le DataFrame avec les colonnes "Entreprise" et les différentes catégories.
        seuil (int): Le nombre minimum de catégories pour qu'une entreprise soit sélectionnée.

    Returns:
        pd.DataFrame: Un sous-DataFrame avec seulement les entreprises concernées.
    """
    # On exclut la colonne "Entreprise" pour le comptage
    categories = df.columns[1:]

    # Pour chaque ligne, on compte le nombre de ✓ dans les colonnes catégorie
    df["Nombre de catégories"] = df[categories].apply(lambda row: row.eq("✓").sum(), axis=1)

    # On filtre les entreprises qui sont dans au moins `seuil` catégories
    df_filtré = df[df["Nombre de catégories"] >= seuil].copy()

    # Optionnel : trier par le nombre décroissant de catégories
    df_filtré.sort_values("Nombre de catégories", ascending=False, inplace=True)

    return df_filtré
df = pd.read_csv("/Users/cosima/Desktop/Finance durable/projet fin durable/ESG_Indices_and_Labels_Ticker.csv")
resultat = entreprises_multiples_categories(df)

In [135]:
resultat

,Entreprise,MSCI World Low Carbon SRI,MSCI Europe Low Carbon SRI,MSCI World ESG Leaders,FTSE4Good,S&P Clean Energy,S&P 500 ESG Leaders,S&P SmallCap 600,S&P MidCap 400,S&P EUROUSA 50,S&P 500 Equal Weight ESG+,S&P Water Decrement,S&P Water Index,CSR Hub Better World Companies AND 100 Low Carbon Pioneers,Sustainalytics top ESG,Nombre de catégories
77,MA,✓,NaN,NaN,✓,NaN,✓,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
10,NVDA,✓,NaN,NaN,✓,NaN,✓,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
129,V,✓,NaN,NaN,✓,NaN,✓,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
124,UNH,NaN,NaN,NaN,✓,NaN,✓,NaN,NaN,NaN,✓,NaN,NaN,NaN,NaN,3
123,GOOGL,✓,NaN,NaN,✓,NaN,✓,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,MSFT,✓,NaN,NaN,✓,NaN,✓,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
93,AWK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,✓,✓,NaN,NaN,2
127,TSLA,✓,NaN,NaN,NaN,NaN,✓,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
113,SVT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,✓,✓,NaN,NaN,2
2,WTRG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,✓,✓,NaN,NaN,2


AWK


# Pays avec les meilleurs scores esg selon différents critères ESG
### Décision d'investissement obligataire

In [ ]:
import requests
import csv
import time

# Liste des indicateurs ESG à récupérer
indicators = {
    "IC.BUS.EASE.XQ": "Ease of doing business rank (1=most business-friendly regulations)",
    "EG.FEC.RNEW.ZS": "Renewable energy (% of total energy)",
    "CC.EST":"Control of corruption",
    "NY.GDP.PCAP.KD.ZG": "GDP per capita growth (annual %)",
    "EG.USE.COMM.FO.ZS": "Fossil fuel energy consumption (% of total)",
    "FR.INR.RISK":"Risk premium on lending (lending rate minus treasury bill rate, %)",
    }

# Liste des pays de l'OCDE (codes ISO3)
oecd_countries = [
    "AUS", "AUT", "BEL", "CAN", "CHL", "COL", "CRI", "CZE", "DNK", "EST",
    "FIN", "FRA", "DEU", "GRC", "HUN", "ISL", "IRL", "ISR", "ITA", "JPN",
    "KOR", "LVA", "LTU", "LUX", "MEX", "NLD", "NZL", "NOR", "POL", "PRT",
    "SVK", "SVN", "ESP", "SWE", "CHE", "TUR", "GBR", "USA"
]

# Récupérer nom du pays
def get_country_name(code):
    r = requests.get(f"https://api.worldbank.org/v2/country/{code}?format=json")
    if r.status_code == 200:
        try:
            return r.json()[1][0]['name']
        except:
            return code
    return code

# Récupérer dernière valeur disponible pour un indicateur
def get_latest_value(country_code, indicator_code):
    url = f"https://api.worldbank.org/v2/country/{country_code}/indicator/{indicator_code}?format=json&per_page=100"
    r = requests.get(url)
    if r.status_code != 200:
        return None
    try:
        data = r.json()
        if len(data) < 2 or not isinstance(data[1], list):
            return None
        for entry in data[1]:
            if entry["value"] is not None:
                return entry["value"]
    except:
        return None
    return None

# Main
def main():
    output_file = "esg_oecd_worldbank.csv"

    with open(output_file, mode="w", newline="", encoding="utf-8") as csv_file:
        writer = csv.writer(csv_file)
        header = ["Country"] + list(indicators.values())
        writer.writerow(header)

        for code in oecd_countries:
            country_name = get_country_name(code)
            row = [country_name]
            for ind_code in indicators.keys():
                value = get_latest_value(code, ind_code)
                row.append(value)
                time.sleep(0.2)
            writer.writerow(row)
            print(f"✔ {country_name} done.")

    print(f"\nFichier généré : {output_file}")

if __name__ == "__main__":
    main()

✔ Australia done.
✔ Austria done.
✔ Belgium done.
✔ Canada done.
✔ Chile done.
✔ Colombia done.
✔ Costa Rica done.
✔ Czechia done.
✔ Denmark done.
✔ Estonia done.
✔ Finland done.
✔ France done.
✔ Germany done.
✔ Greece done.
✔ Hungary done.
✔ Iceland done.
✔ Ireland done.
✔ Israel done.
✔ Italy done.
✔ Japan done.
✔ Korea, Rep. done.
✔ Latvia done.
✔ Lithuania done.
✔ Luxembourg done.
✔ Mexico done.
✔ Netherlands done.
✔ New Zealand done.
✔ Norway done.
✔ Poland done.
✔ Portugal done.
✔ Slovak Republic done.
✔ Slovenia done.
✔ Spain done.
✔ Sweden done.
✔ Switzerland done.
✔ Turkiye done.
✔ United Kingdom done.
✔ United States done.

Fichier généré : esg_oecd_worldbank.csv


In [35]:

# Charger les données
df = pd.read_csv("esg_oecd_worldbank.csv")

# Nettoyer : s'assurer que les colonnes numériques sont bien en float
for col in df.columns[1:]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Fonction pour afficher les 10 meilleurs pays par indicateur
def afficher_top_5(df):
    for col in df.columns[1:]:
        print(f"\nTop 5 pays pour:")
        top5 = df[["Country", col]].dropna().sort_values(by=col, ascending=False).head(5)
        print(top5.to_string(index=False))

# Lancer l’analyse
afficher_top_5(df)



Top 5 pays pour:
   Country  Ease of doing business rank (1=most business-friendly regulations)
    Greece                                                                  79
Costa Rica                                                                  74
Luxembourg                                                                  72
  Colombia                                                                  67
    Mexico                                                                  60

Top 5 pays pour:
Country  Renewable energy (% of total energy)
Iceland                                  82.4
 Norway                                  61.4
 Sweden                                  57.9
Finland                                  50.2
 Latvia                                  44.0

Top 5 pays pour:
    Country  Control of corruption
    Denmark               2.376053
    Finland               2.221517
     Norway               2.113522
New Zealand               2.084095
     Sweden          

In [63]:
import csv

# Chemin vers ton fichier CSV (ex. : 'overall_score.csv')
csv_path = "SDR-2024-overall-score.csv"


evolutions = []

with open(csv_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)  # En-tête : ISO, Country, 2000, ..., 2023
    
    for row in reader:
        if not row or len(row) < 2:
            continue
        if row[1] == "Country" or row[0].startswith("Sustainable"):
            continue

        iso = row[0]
        country = row[1]

        try:
            # Convertir les données de 2000 à 2023 en flottants
            scores = [float(value) for value in row[2:] if value.strip()]
            
            # Calcul de la moyenne sur la période 2000-2023
            average_score = sum(scores) / len(scores)
            evolutions.append((country, average_score))
        except (ValueError, IndexError):
            print(f"Erreur avec les données de {country}.")
            continue

# Tri des pays selon la moyenne la plus haute
evolutions.sort(key=lambda x: x[1], reverse=True)

# Affichage du top 5 des pays avec la moyenne la plus haute
print("Top 5 des pays avec la moyenne la plus haute de score (2000 → 2023) :")
for i, (country, avg_score) in enumerate(evolutions[:5], 1):
    print(f"{i}. {country} : {avg_score:.2f}")



Top 5 des pays avec la moyenne la plus haute de score (2000 → 2023) :
1. Germany : 80.27
2. France : 79.42
3. United Kingdom : 79.37
4. Japan : 77.70
5. Canada : 76.70


Critères d'inclusion pour les obligations d'Etats: 
- Ease of doing business rank (1=most business-friendly regulations)
- Renewable energy (% of total energy)
- Control of corruption
- GDP per capita growth (annual %)
- SDR score result

Donc on garde : Greece, Costa Rica, Turkiye (non), Sweden, Norway, Finland, Denmark, Iceland, Latvia, Luxembourg, Colombia, France, Germany, United Kingdom, Canada

Critères d'exclusion : 
- Fossil fuel energy consumption (% of total)
- Risk premium on lending (lending rate minus treasury bill rate, %)

On exclut : Hungary, Australia, United States, Czechia, New Zealand, Israel, Netherlands, Japan, Mexico, Poland



- IRLTLT01GRM156N : Long-Term Government Bond Yields: 10-Year: Main (Including Benchmark) for Greece
- CRIIRLTLT01STM : Long-Term Government Bond Yields: 10-Year: Main (Including Benchmark) for Costa Rica
- IRLTLT01SEM156N : Long-Term Government Bond Yields: 10-Year: Main (Including Benchmark) for Sweden
- IRLTLT01NOM156N : Long-Term Government Bond Yields: 10-Year: Main (Including Benchmark) for Norway
- IRLTLT01FIM156N : Long-Term Government Bond Yields: 10-Year: Main (Including Benchmark) for Finland 
- IRLTLT01DKM156N : Long-Term Government Bond Yields: 10-Year: Main (Including Benchmark) for Denmark
- IRLTLT01ISM156N : Long-Term Government Bond Yields: 10-Year: Main (Including Benchmark) for Iceland
- LVAIRLTLT01STM : Long-Term Government Bond Yields: 10-Year: Main (Including Benchmark) for Latvia
- IRLTLT01LUM156N : Long-Term Government Bond Yields: 10-Year: Main (Including Benchmark) for Luxembourg
- COLIRLTLT01STM : Long-Term Government Bond Yields: 10-Year: Main (Including Benchmark) for Colombia
- IRLTLT01FRM156N : Long-Term Government Bond Yields: 10-Year: Main (Including Benchmark) for France
- IRLTLT01DEM156N : Germany
- IRLTLT01GBM156N : UK
- IRLTLT01CAM156N : Canada

In [64]:
import os
import numpy as np
import pandas as pd
from scipy import stats
from fredapi import Fred

class FredDataCollector:
    """Classe pour télécharger, nettoyer et sauvegarder des données FRED."""

    FRED_TICKERS = [
        "IRLTLT01GRM156N",  # Greece
        "CRIIRLTLT01STM",   # Costa Rica
        "IRLTLT01SEM156N",  # Sweden
        "IRLTLT01NOM156N",  # Norway
        "IRLTLT01FIM156N",  # Finland
        "IRLTLT01DKM156N",  # Denmark
        "IRLTLT01ISM156N",  # Iceland
        "LVAIRLTLT01STM",   # Latvia
        "IRLTLT01LUM156N",  # Luxembourg
        "COLIRLTLT01STM",   # Colombia
        "IRLTLT01FRM156N",  # France
        "IRLTLT01DEM156N", #Germany
        "IRLTLT01GBM156N", #UK
        "IRLTLT01CAM156N", #Canada
    ]

    def __init__(self, api_key, start_date="2000-01-01", end_date="2024-12-31", output_dir="macro_data"):
        self.start_date = start_date
        self.end_date = end_date
        self.output_dir = output_dir
        self.fred = Fred(api_key=api_key)
        os.makedirs(output_dir, exist_ok=True)

    def download_data(self):
        """Télécharge les séries FRED et retourne un DataFrame consolidé."""
        print("Téléchargement des données...")
        return pd.DataFrame({
            ticker: self.fred.get_series(ticker, self.start_date, self.end_date)
            for ticker in self.FRED_TICKERS
        })

    @staticmethod
    def handle_missing_values(df):
        """Gère les valeurs manquantes avec forward-fill et backward-fill."""
        missing_before = df.isna().sum().sum()
        df = df.ffill().bfill()
        missing_after = df.isna().sum().sum()
        print(f"Valeurs manquantes : {missing_before} → {missing_after}")
        return df

    @staticmethod
    def remove_outliers(df):
        """Supprime les outliers via Z-score (> 3σ) puis gère les NaN."""
        print("Suppression des valeurs aberrantes...")
        z_scores = np.abs(stats.zscore(df, nan_policy='omit'))
        df[z_scores > 3] = np.nan
        return FredDataCollector.handle_missing_values(df)

    def save_to_csv(self, df, filename="obli_souveraines.csv"):
        """Sauvegarde le DataFrame sous forme de fichier CSV."""
        filepath = os.path.join(self.output_dir, filename)
        df.to_csv(filepath)
        print(f"Données sauvegardées dans : {filepath}")

    def run(self):
        """Pipeline complet : collecte, nettoyage, sauvegarde."""
        df = self.download_data()
        df_clean = self.remove_outliers(df)
        self.save_to_csv(df_clean)
        return df_clean

# Exemple d'utilisation
if __name__ == "__main__":
    api_key = "41c3708cd6c3fc889a399c48e21b2791"
    collector = FredDataCollector(api_key)
    data = collector.run()


Téléchargement des données...
Suppression des valeurs aberrantes...
Valeurs manquantes : 237 → 0
Données sauvegardées dans : macro_data/obli_souveraines.csv


In [65]:
import pandas as pd

def top_5_pays_2024(csv_file):
    # Charger le fichier CSV
    df = pd.read_csv(csv_file, parse_dates=[0], index_col=0)
    
    # Filtrer les données pour l'année 2024
    df_2024 = df[df.index.year == 2024]
    
    # Sélectionner la dernière ligne (taux de décembre 2024) pour chaque pays
    dernier_taux_2024 = df_2024.iloc[-1]
    
    # Trier les pays en fonction des taux les plus élevés
    top_5_pays = dernier_taux_2024.sort_values(ascending=False).head(5)
    
    return top_5_pays

# Exemple d'utilisation
# Remplacez 'votre_fichier.csv' par le chemin de votre fichier CSV
top_5 = top_5_pays_2024('/Users/cosima/Desktop/Finance durable/projet fin durable/macro_data/obli_souveraines.csv')
print(top_5)


COLIRLTLT01STM     11.720000
CRIIRLTLT01STM      6.420000
IRLTLT01GBM156N     4.434500
IRLTLT01NOM156N     3.598889
IRLTLT01CAM156N     3.161500
Name: 2024-12-01 00:00:00, dtype: float64


In [66]:
import pandas as pd

def pays_avec_volatilite_minimale(csv_file):
    # Charger le fichier CSV
    df = pd.read_csv(csv_file, parse_dates=[0], index_col=0)
    
    # Calculer la volatilité (écart-type) pour chaque pays
    volatilites = df.std()
    
    # Trier les pays par volatilité croissante (les moins volatils en premier)
    volatilites_triees = volatilites.sort_values(ascending=True)
    
    # Renvoyer les pays avec la volatilité la plus faible
    return volatilites_triees.head(5)  # Renvoie les 5 pays les moins volatils

# Exemple d'utilisation
# Remplacez 'votre_fichier.csv' par le chemin de votre fichier CSV
top_5_pays = pays_avec_volatilite_minimale('/Users/cosima/Desktop/Finance durable/projet fin durable/macro_data/obli_souveraines.csv')
print(top_5_pays)


CRIIRLTLT01STM     1.127915
IRLTLT01CAM156N    1.438251
IRLTLT01ISM156N    1.556080
IRLTLT01GBM156N    1.581694
IRLTLT01NOM156N    1.616208
dtype: float64


Pays dans lesquels investir : Canada, Costa Rica, Iceland, UK, Norway, Colombia
#____________________________

# Fonds à impact

In [136]:
import pandas as pd

# Liste des fonds avec le label Greenfin
greenfin_funds = [
    "Allianz Impact Green Bond", "Allianz Green Bond", "FPCI AMUNDI PRIVATE EQUITY TRANSITION JUSTE",
    "AMUNDI RESPONSIBLE INVESTING – IMPACT GREEN BONDS", "Ircantec Green Bonds Amundi AM", 
    "AMUNDI FUNDS IMPACT GREEN BONDS", "Amundi Funds Euro Corporate Short Term Impact Green Bonds", 
    "AMUNDI CREDIT GREEN BONDS", "AMUNDI KBI ACTIONS MONDE ENVIRONNEMENT", "LCL ACTIONS MONDE ENVIRONNEMENT", 
    "SG OBLIGATIONS VERTES", "Amundi Obligations Vertes ESR", "Amundi Responsible Investing - Impact Euro Corporate Green Bond", 
    "GFI SYMBIOSE", "Andera Smart Infra 1", "Andera Smart Infra 2", "Stratégie Obligations Durables", 
    "Transition Evergreen", "AXA Sélection Obligations Monde", "Ircantec Green Bonds", "AXA Horizon Durable", 
    "AGIPI Obligations Monde", "AXA WF ACT Dynamic Green Bonds", "AXA WF ACT Green Bonds", "iShares Green Bond Fund", 
    "BNP Paribas Funds Green Bond", "Multipar Green Bond", "Conquest Infrastructure SLP Sustainable II", 
    "Covéa Obligations", "CM-AM Global Climate Change", "CM-AM Green Bonds", "Fonds de Modernisation Ecologique des Transports", 
    "Paris fonds verts", "DORVAL EUROPEAN CLIMATE INITIATIVE", "Ecofi Agir pour le climat", "PEARL Infrastructure Capital", 
    "EDRAM Bridge IV Europe Senior", "PEARL Infrastructure Capital II", "Egamo Obligation Euro", 
    "Eiffel Infrastructures Vertes", "Allianz Transition Energétique", "Eiffel Gaz Vert", "MAIF Rendement Vert", 
    "ETE - Energy Transition Europe", "Famae Impact I", "Federal Global Green Bonds", "GFF MAIF Forêts", 
    "GFI France Valley Forêts I", "GFI France Valley Patrimoine", "France Valley Entrepreneurs", 
    "France Valley Foncière Europe", "GFI France Valley Forêts II", "GFI France Valley Forêts III", 
    "GFI France Valley Forêts IV", "GFI France Valley Forêts V", "GFI France Valley Forêts VI", 
    "GFI France Valley Forêts VII", "GFI France Valley Forêts VIII", "France Valley Revenu Europe I", 
    "France Valley Revenu Europe II", "France Valley Revenu Europe III", "France Valley Revenu Europe IV", 
    "France Valley Revenu Europe V", "France Valley Revenu Europe VI", "France Valley Revenu Europe VII", 
    "France Valley Revenu Europe IX", "France Valley - Investissement Forêts Europe IR I", 
    "France Valley - Investissement Forêts Europe IR II", "GFI France Valley Forêts IX", "GFI France Valley Forêts X", 
    "Glennmont Clean Energy Fund IV", "GS Corporate Green Bond", "GS Green Bond", "GS Green Bond Short Duration", 
    "GS Sovereign Green Bond", "HSBC EE Actions Europe Transition Verte", "HSBC RIF Europe Equity Green Transition", 
    "Fonds Développement Filière Bois", "CA Transitions B", "Tocqueville environnement ISR", "Tutélaire Actions ISR", 
    "La Française LUX – Inflection Point Carbon Impact Global", "Mandarine Global Transition", 
    "Afer Actions Environnement", "MAM Impact Green Bonds", "MERIDIAM GREEN IMPACT GROWTH FUND", 
    "Mirova Energy Transition 6", "Mirova Energy Transition 5", "Mirova Climate Equity", "MIROVA EUROFIDEME 4", 
    "Mirova Eurofideme 3", "Mirova Europe Environmental Equity Fund", "Mirova Global Green Bond Fund", 
    "Mirova Environment Accelerating Capital", "Mirova Europe Environnement", "Monceau Multi Gestion Finance Verte", 
    "M Climate Solutions", "Green Yield Infra Debt Fund 2", "Purple Private Debt SCS RAIF - Green Yield Infra Debt Fund", 
    "Purple Private Debt SCS RAIF - Essential Infra Debt Fund", "OCCTE OCCIGEN", "ODDO BHF Green Bond", 
    "OFI Act4 Green Future", "Capenergie IV", "Infragreen II", "Infragreen III", "Infragreen IV", "Infragreen V", 
    "ISF BlueOrchard Emerging Markets Climate Bond", "Schroder ISF Global Energy Transition", "F2E", 
    "Predirec ENR 2030", "Predirec ENR 2", "Sienna Obligations Vertes ISR (ancien HGA Obligations Vertes)", 
    "Moorea Fund – Sustainable climate action", "GFI CoeurForest", "SWEN Impact Fund for Transition 2 (SWIFT 2)", 
    "SWEN Impact Fund for Transition 2 (SWIFT 3)", "Sycomore Europe Eco Solutions", "Prestige Luxembourg - UZES Biodiversité", 
    "Zencap European Sustainable Transition II FPS", "Zencap Energy Transition Debt III M"
]

ISR_funds = [
    "CANDRIAM SUSTAINABLE EQUITY CIRCULAR ECONOMY",
    "FCPE GROUPAMA EPARGNE ET RETRAITE MONETAIRE",
    "AXA ISR EUROPE ACTIONS",
    "FCPE GROUPAMA EPARGNE ET RETRAITE PERSPECTIVE TRESORERIE",
    "Eres Select ISR Environnement",
    "SG Mirova Actions Monde Climat",
    "Covéa Flexible ISR",
    "Ostrum SRI Euro Sovereign Bonds",
    "TOCQUEVILLE EURO EQUITY ISR",
    "Euro Short Duration SRI",
    "AXA Valeurs Euro",
    "AXA France Small Cap",
    "AGIPI Actions Monde",
    "LBPAM ISR ACTIONS FOCUS EURO",
    "LBPAM ISR PROFIL 50",
    "LBPAM ISR PROFIL 25",
    "AXA Court Terme",
    "Ecofi Trésorerie",
    "Norden SRI",
    "Palatine Monétaire Standard",
    "SG AMUNDI ACTIONS EUROPE ENVIRONNEMENT ISR",
    "AXA Trésor Court Terme",
    "AXA France Opportunités",
    "LBPAM ISR ACTIONS EUROPE MONDE",
    "Nouvelle Stratégie 50",
    "Ecofi Convictions Monde",
    "Ofi Invest ESG Monétaire Court Terme",
    "Mirova Actions Europe",
    "AXA IM EURO LIQUIDITY SRI",
    "Ofi Invest ESG Equity Climate Change",
    "AXA Euro Valeurs Responsables",
    "ALM ACTIONS ZONE EURO ISR",
    "LCL ACTIONS EURO ODD RESPONSABLE",
    "LCL ACTIONS ETATS-UNIS RESPONSABLE",
    "Ofi Invest ISR Monetaire CT",
    "CM-AM EURO EQUITIES",
    "LCL ACTIONS DEVELOPPEMENT DURABLE",
    "GROUPAMA TRÉSORERIE",
    "AMUNDI VALEURS DURABLES",
    "AMUNDI ACTIONS EUROPE RESPONSABLE",
    "LCL ACTIONS USA RESPONSABLE",
    "Allianz Multi Tempéré ISR",
    "ALM OBLIG EURO ISR",
    "LBPAM ISR AVENIR EURO",
    "Diversis Dynamique",
    "Tego Dynamique ISR",
    "AMUNDI EURO LIQUIDITY-RATED RESPONSIBLE",
    "Monceau Ethique",
    "ALM CRÉDIT EURO ISR",
    "Label Europe Actions",
    "Label Euro Obligations",
    "BNP Paribas Actions Rendement ISR",
    "Ostrum SRI Money",
    "Ecofi Avenir Plus",
    "Gaspal Obligations Court Terme",
    "CM-AM OBLIGATIONS ISR",
    "CM-AM SOLIDAIRE TEMPERE ISR",
    "UZES MONDE PLI",
    "Ofi Invest ISR Credit Bonds Euro 1-3",
    "Portzamparc Opportunités ISR",
    "AMUNDI EURO LIQUIDITY SHORT TERM RESPONSIBLE",
    "HYMNOS ISR",
    "CARDIF BNPP AM Sustainable Europe Equity",
    "TAILOR ACTIONS AVENIR ISR",
    "Ofi Invest ISR Actions Europe",
    "CARDIF BNPP AM Sustainable Euro Equity",
    "BSO France",
    "Mansartis Asie ISR",
    "UZES PEA",
    "Diversis",
    "ALM ACTIONS EUROPE ISR",
    "Mansartis Investissements",
    "HARMONIE MUTUELLE EMPLOIS France",
    "Allianz Securicash SRI",
    "BNP PARIBAS ACTIONS PATRIMOINE ISR",
    "Mirova Obli Euro",
    "AXA ACT Carbon Offset Eurobloc Equity QI",
    "AXA ACT Carbon Offset Equity QI",
    "CARDIF BNPP AM Euro Paris Climate Aligned",
    "BNP PARIBAS OBLI ISR",
    "Swiss Life Funds (F) ESG Money Market Euro",
    "BNP PARIBAS RENDACTIS ISR",
    "Villiers Actions Europe AGI",
    "BNP PARIBAS ACTIONS MONDE ISR",
    "Schelcher Convertible",
    "Carmignac Emergents",
    "AMUNDI ACTIONS USA RESPONSABLE",
    "EdR SICAV – Euro Sustainable Credit",
    "AXA Optimal Income",
    "Ostrum SRI Euro Bonds 5-7",
    "Mansartis Japon ISR",
    "BNP PARIBAS SUSTAINABLE INVEST 3M",
    "Ecofi Trajectoires Durables",
    "BFT FRANCE MONETAIRE COURT TERME ISR",
    "VIVACCIO ISR ACTIONS",
    "BACH MONETAIRE",
    "GROUPAMA ENTREPRISES",
    "LBPAM ISR OBLI REVENUS",
    "LBPAM ISR ACTIONS SMALL CAP EURO",
    "ECHIQUIER MAJOR SRI GROWTH EUROPE",
    "BNP Paribas Actions Croissance ISR",
    "Allianz Valeurs durables",
    "BFT FRANCE FUTUR ISR CLIMAT",
    "Palatine Planète",
    "Mandarine Equity Income",
    "CPR CAS RESPONSIBLE",
    "SG AMUNDI MONETAIRE ISR",
    "Portzamparc Europe PME ISR",
    "AMUNDI ACTIONS EURO RESPONSABLE",
    "AGIPI Grandes Tendances",
    "AGIPI MONDE DURABLE",
    "CPR USA RESPONSABLE",
    "LFR EURO DEVELOPPEMENT DURABLE",
    "Ostrum SRI Cash",
    "Immobilier 21",
    "Sextant PME",
    "BFT AUREUS ISR",
    "AGIPI Actions Europe",
    "AGIPI Obligations Inflation",
    "Mirova Emploi France",
    "Actions solidaires",
    "ECHIQUIER ARTY SRI",
    "LBPAM ISR ACTIONS FOCUS France",
    "LBPAM ISR OBLI MOYEN TERME",
    "TOCQUEVILLE TECHNOLOGY ISR",
    "LBPAM ISR OBLI EUROPE",
    "LBPAM ISR OBLI LONG TERME",
    "LBPAM ISR MULTI ACTIONS EURO",
    "LBPAM ISR MULTI ACTIONS MONDE",
    "LBPAM ISR THEMATIQUES DURABLES",
    "TONI ACTIONS ISR 100",
    "Arkéa Actions Euro",
    "Ecofi Agir pour le Climat",
    "BNP Paribas Valeurs Euro ISR",
    "Mandarine Opportunités",
    "OPCI FRANCEUROPE IMMO",
    "Carac Actions Euro",
    "BNP PARIBAS AQUA",
    "GROUPAMA MONÉTAIRE",
    "MAIF Actions Transition Climat (dédié CPRAM)",
    "GROUPAMA EURO CREDIT IC",
    "CPR EUROLAND RESPONSABLE",
    "CPR MONETAIRE RESPONSABLE",
    "Ofi Invest ISR Croissance Durable",
    "AMUNDI RESPONSIBLE INVESTING - EUROPEAN CREDIT",
    "Ostrum SRI Money 6M",
    "Euro Credit SRI",
    "BFT CREDIT 6 MOIS ISR",
    "AMUNDI RENDEMENT PLUS RESPONSABLE",
    "AFER Diversifie Durable ISR",
    "Ostrum SRI Money Plus",
    "ECHIQUIER SHORT TERM CREDIT SRI",
    "AMUNDI ACTIONS FRANCE RESPONSABLE",
    "Arkéa Crédit Court Terme",
    "ECHIQUIER POSITIVE IMPACT EUROPE",
    "LBPAM ISR ACTIONS SOLIDAIRE",
    "Ostrum SRI Cash Plus",
    "Amplegest Pricing Power",
    "Groupama Euro Active Equity",
    "Palatine Europe Sustainable Employment",
    "CPR OBLIG 12 MOIS RESPONSABLE",
    "Ofi Invest ESG Allocation Flexible",
    "DNCA SRI Euro Quality",
    "AXA IM EURO 6M",
    "OPPCI VIVALDI",
    "GROUPAMA EURO GOVIES",
    "AGIPI Actions Emergents",
    "BNP PARIBAS ACTIONS PME ISR",
    "Schelcher Short Term",
    "Ostrum SRI Cash A1P1",
    "Epargne éthique obligations",
    "Epargne Ethique Monétaire",
    "LBPAM ISR STRATEGIE PEA 2",
    "Swiss Life Funds (F) ESG Short Term Euro",
    "OPCIMMO",
    "SG ACTIONS EURO RENDEMENT ISR",
    "AMUNDI ULTRA SHORT TERM BOND RESPONSIBLE",
    "Ofi Invest ISR Valeurs Europe",
    "AXA SELECTIV'IMMO",
    "CPR GLOBAL ALLOCATION RESPONSIBLE",
    "Schelcher Convertible Mid Cap",
    "Sanso Essentiel",
    "Pilotage Sélection Durable",
    "DECISIEL ISR OBLIGATAIRE",
    "SYCOMORE SÉLECTION CRÉDIT",
    "Ostrum SRI Euro Bonds 3-5",
    "LBPAM ISR AVENIR EPARGNE",
    "Trecento Santé ISR",
    "Ostrum SRI Crossover",
    "AGIPI CLIMAT",
    "Generali Trésorerie ISR",
    "Ofi Invest ESG Monétaire",
    "Amplegest Pricing Power US",
    "BNP PARIBAS DEVELOPPEMENT HUMAIN",
    "LBPAM ISR ACTIONS EUROPE",]

# Création d'un DataFrame avec les fonds et l'indication du label Greenfin
fund_names = greenfin_funds + ISR_funds
greenfin_labels = ["✓" if fund in greenfin_funds else "" for fund in fund_names]

df = pd.DataFrame({
    'Fund Name': fund_names,
    'Greenfin Label': ["✓" if fund in greenfin_funds else "" for fund in fund_names],
    'ISR Label': ["✓" if fund in ISR_funds else "" for fund in fund_names]
})

# Save updated CSV
file = "Fonds_greenfin_ISR.csv"
df.to_csv(file, index=False)

In [80]:
import pandas as pd

# Data provided
data = [
    ["AMUNDI ASSET MANAGEMENT", "Amundi Label ISR Actions Europe UCITS ETF", "ETF - actions européennes", "FR001400SDI8"],
    ["AMUNDI ASSET MANAGEMENT", "Amundi Label ISR Actions Japon UCITS ETF", "ETF - actions internationales", "FR001400SDJ6"],
    ["AMUNDI ASSET MANAGEMENT", "Amundi Label ISR Actions Marchés Emergents UCITS ETF", "ETF - actions internationales", "FR001400SDM0"],
    ["AMUNDI ASSET MANAGEMENT", "Amundi Label ISR Actions Monde UCITS ETF", "ETF - actions internationales", "FR001400SDM0"],
    ["AMUNDI ASSET MANAGEMENT", "Amundi Label ISR Actions USA UCITS ETF", "ETF - actions US", "FR001400SDN8"],
    ["COVEA FINANCE", "Covéa Flexible ISR", "Diversifié", "Europe", "FR0000002164"],
    ["LA BANQUE POSTALE ASSET MANAGEMENT", "LBPAM ISR ACTIONS FOCUS EURO", "Actions", "Europe", "FR0000285884"],
    ["AMUNDI ASSET MANAGEMENT", "AMUNDI ACTIONS EUROPE RESPONSABLE", "Actions", "Europe", "FR0000991432"],
    ["AXA INVESTMENT MANAGERS", "Label Europe Actions", "Actions", "Europe", "FR0007073713"],
    ["BNP PARIBAS ASSET MANAGEMENT", "BNP Paribas Actions Rendement ISR", "Actions", "Europe", "FR0007074208"],
    ["BNP PARIBAS ASSET MANAGEMENT", "CARDIF BNPP AM Sustainable Europe Equity", "Actions", "Europe", "FR0007457866"],
    ["BNP PARIBAS ASSET MANAGEMENT", "CARDIF BNPP AM Sustainable Euro Equity", "Diversifié", "Europe", "FR0007474861"],
    ["BNP PARIBAS ASSET MANAGEMENT", "BNP PARIBAS ACTIONS PATRIMOINE ISR", "Actions", "Europe", "FR0010028910"],
    ["BNP PARIBAS ASSET MANAGEMENT", "CARDIF BNPP AM Euro Paris Climate Aligned", "Diversifié", "Europe", "FR0010076869"],
    ["BNP PARIBAS ASSET MANAGEMENT", "BNP PARIBAS OBLI ISR", "Obligations", "Europe", "FR0010076893"],
    ["BNP PARIBAS ASSET MANAGEMENT", "BNP PARIBAS RENDACTIS ISR", "Actions", "Europe", "FR0010102095"],
    ["AXA INVESTMENT MANAGERS", "AXA Optimal Income", "Diversifié", "Europe", "FR0010188342"],
    ["BNP PARIBAS ASSET MANAGEMENT", "BNP PARIBAS SUSTAINABLE INVEST 3M", "Monétaire", "Europe", "FR0010209916"],
    ["ECOFI", "Ecofi Trajectoires Durables", "Actions", "Europe", "FR0010214213"],
    ["LA BANQUE POSTALE ASSET MANAGEMENT", "ECHIQUIER MAJOR SRI GROWTH EUROPE", "Actions", "Europe", "FR0010321828"],
    ["MANDARINE GESTION", "Mandarine Equity Income", "Actions", "Europe", "FR0010396382"],
    ["OSTRUM ASSET MANAGEMENT", "Ostrum SRI Cash", "Monétaire", "Europe", "FR0010529743"],
    ["LA BANQUE POSTALE ASSET MANAGEMENT", "ECHIQUIER ARTY SRI", "Diversifié", "Europe", "FR0010611293"],
    ["CPR ASSET MANAGEMENT", "MAIF Actions Transition Climat (dédié CPRAM)", "Actions", "Europe", "FR0010696815"],
    ["OSTRUM ASSET MANAGEMENT", "Ostrum SRI Money 6M", "Monétaire", "Europe", "FR0010750984"],
    ["LA BANQUE POSTALE ASSET MANAGEMENT", "ECHIQUIER SHORT TERM CREDIT SRI", "Obligations", "Europe", "FR0010839282"],
    ["LA BANQUE POSTALE ASSET MANAGEMENT", "ECHIQUIER POSITIVE IMPACT EUROPE", "Actions", "Europe", "FR0010863688"],
    ["SYCOMORE ASSET MANAGEMENT", "SYCOMORE SÉLECTION CRÉDIT", "Obligations", "Europe", "FR0011288513"],
    ["OSTRUM ASSET MANAGEMENT", "Ostrum SRI Crossover", "Obligations", "Europe", "FR0011350677"],
    ["BNP PARIBAS ASSET MANAGEMENT", "BNP PARIBAS MOIS ISR", "Monétaire", "Europe", "FR0011482686"],
    ["CREDIT MUTUEL ASSET MANAGEMENT", "La Française Sub Debt", "Obligations", "Europe", "FR0011766401"],
    ["BNP PARIBAS ASSET MANAGEMENT", "BNP Paribas Actions PME ETI ISR", "Actions", "Europe", "FR0013254380"],
    ["PROBTP FINANCE", "REGARD RESPONSABLE FLEXIBLE", "Fonds de fonds", "Europe", "FR0013420197"],
    ["SG 29 HAUSSMANN", "SG Echiquier Actions Europe Midcap ISR", "Actions", "Europe", "FR0013535523"],
    ["SG 29 HAUSSMANN", "SG Mirova Actions Europe Environnement Durable", "Actions", "Europe", "FR0050000449"],
    ["SYCOMORE ASSET MANAGEMENT", "GIS SRI EUROPEAN EQUITY", "Actions", "Europe", "LU0145456207"],
    ["CANDRIAM", "CANDRIAM EQUITIES L EUROPE INNOVATION", "Actions", "Europe", "LU0344046668"],
    ["MANDARINE GESTION", "Mandarine Unique", "Actions", "Europe", "LU0489687243"],
    ["SYCOMORE ASSET MANAGEMENT", "GIS SRI AGEING POPULATION", "Actions", "Europe", "LU1234787460"],
    ["BNP PARIBAS ASSET MANAGEMENT", "BNP PARIBAS EASY LOW CARBON 100 EUROPE PAB®", "Actions", "Europe", "LU1377382368"],
    ["M&G Investments Limited SA", "M&G (Lux) Pan European Sustain Paris Aligned Fund", "Actions", "Europe", "LU1670716437"],
    ["BNP PARIBAS ASSET MANAGEMENT", "BNP PARIBAS EASY MSCI Europe SRI S-Series PAB 5% Capped", "Actions", "Europe", "LU1753045332"],
    ["BNP PARIBAS ASSET MANAGEMENT", "MULTIPAR ACTIONS EUROPE BAS CARBONE", "Actions", "Europe", "QS0002103MF0"],
    ["BNP PARIBAS ASSET MANAGEMENT", "MULTIPAR ACTIONS SOCIALEMENT RESPONSABLE", "Actions", "Europe", "QS0002105TI4"],
    ["BNP PARIBAS ASSET MANAGEMENT", "MULTIPAR ACTIONS PME ETI ISR", "Actions", "Europe", "QS000212BEQ0"],
    ["BNP PARIBAS ASSET MANAGEMENT", "BNP Paribas Actions Euro ISR", "Actions", "Europe", ""]
]

# Create a dataframe
df = pd.DataFrame(data, columns=["Asset Manager", "Fund Name", "Asset Class", "Region", "ISIN"])

# Display dataframe
df
df = df.drop(columns=["Region","Asset Manager","Asset Class"])

In [81]:
df

,Fund Name,ISIN
0,Amundi Label ISR Actions Europe UCITS ETF,None
1,Amundi Label ISR Actions Japon UCITS ETF,None
2,Amundi Label ISR Actions Marchés Emergents UCI...,None
3,Amundi Label ISR Actions Monde UCITS ETF,None
4,Amundi Label ISR Actions USA UCITS ETF,None
5,Covéa Flexible ISR,FR0000002164
6,LBPAM ISR ACTIONS FOCUS EURO,FR0000285884
7,AMUNDI ACTIONS EUROPE RESPONSABLE,FR0000991432
8,Label Europe Actions,FR0007073713
9,BNP Paribas Actions Rendement ISR,FR0007074208
